# Generate Full Album Video

This notebook is to be run after the all notebooks for the indivudual tracks have been run and the mp4 files are present in the directory.

The notebook creates a short and silent transition clip between consecutive tracks in the album, and then joins all the video clips together to create the full album video file.

In [1]:
from moviepy.editor import *
from matplotlib import pyplot as plt 
import time
import pandas as pd
from tqdm import tqdm
import numpy as np

from aeraudioviz.image import BaseImage, ImageModifiers
from aeraudioviz.audio.feature_utils import add_random_noise_column, add_sine_wave_column, fade_out_between_times, fade_in_between_times

# Create Transition Clips

The transition clips are a smooth fade between the last frame of the track and first frame of the next. Some salt and pepper noise and saturation modifications are also applied throughout the transitions.

## Get First and Last Frame of Each Video

In [2]:
videos = {
    1: 'Aerodactylus - Jetsam Dreams - 01 The Maiden Flight.mp4',
    2: 'Aerodactylus - Jetsam Dreams - 02 Pterosoaring.mp4',
    3: 'Aerodactylus - Jetsam Dreams - 03 In Plain Flight.mp4',
    4: 'Aerodactylus - Jetsam Dreams - 04 Flutter On.mp4',
    5: 'Aerodactylus - Jetsam Dreams - 05 Wintering Grounds.mp4',
    6: 'Aerodactylus - Jetsam Dreams - 06 Alpenglow.mp4',
    7: 'Aerodactylus - Jetsam Dreams - 07 Jettison the Rest.mp4',
    8: 'Aerodactylus - Jetsam Dreams - 08 Sub Rosa Sunburst.mp4'
}

In [3]:
first_frames = {}
last_frames = {}
for i in videos.keys():
    clip = VideoFileClip(videos[i])
    first_frames[i] = clip.get_frame(0)
    last_frames[i] = clip.get_frame(clip.duration - 1/10.)  # close enough to last clip, fails without some buffer

# Generate the Video Clips

In [4]:
t = 1.5  # length of transition clip in seconds

In [5]:
df = pd.DataFrame(index=pd.to_timedelta([0., t], unit='s'))
df['transition'] = 1.
df = df.resample('41.666666L').mean()
df.fillna(1, inplace=True)
df['noise'] = df['transition'].copy()

df = fade_in_between_times(df, 'transition', 0, t) 
df = fade_in_between_times(df, 'noise', 0, 2.*t/3.) 
df = fade_out_between_times(df, 'noise', 2.*t/3., t) 

In [6]:
def generate(output_path, first_frame, last_frame, feature_time_series):
    frames = []
    for idx, row in tqdm(feature_time_series.iterrows(), total=len(feature_time_series)):
        last_frame_fraction = row['transition']
        first_frame_fraction = 1. - last_frame_fraction
        frame = (first_frame * first_frame_fraction + last_frame * last_frame_fraction).astype(np.uint8)
        frame = ImageModifiers.apply_salt_and_pepper_noise(frame, noise_ratio=.0001)
        frame = ImageModifiers.apply_saturation_multiplication(frame, saturation_factor=1.- row['noise'] * .6)
        frames.append(frame)
    clip = ImageSequenceClip(frames, fps=24)
    clip.write_videofile(output_path, codec="libx264")
    print(f"Video written successfully to {output_path}.")

In [7]:
for i in range(1, 8):
    start = last_frames[i]
    end = first_frames[i + 1]
    generate('transition{}.mp4'.format(str(int(i))), start, end, df)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 37/37 [00:01<00:00, 26.34it/s]


Moviepy - Building video transition1.mp4.
Moviepy - Writing video transition1.mp4



Moviepy - Done !
Moviepy - video ready transition1.mp4
Video written successfully to transition1.mp4.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 37/37 [00:01<00:00, 26.15it/s]


Moviepy - Building video transition2.mp4.
Moviepy - Writing video transition2.mp4



...

Moviepy - Building video transition6.mp4.
Moviepy - Writing video transition6.mp4



Moviepy - Done !
Moviepy - video ready transition6.mp4
Video written successfully to transition6.mp4.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 37/37 [00:01<00:00, 26.09it/s]


Moviepy - Building video transition7.mp4.
Moviepy - Writing video transition7.mp4



Moviepy - Done !
Moviepy - video ready transition7.mp4
Video written successfully to transition7.mp4.


# Join All Video Files Into Full Album Video

In [8]:
video_files = [
    'Aerodactylus - Jetsam Dreams - 01 The Maiden Flight.mp4',
    'transition1.mp4',
    'Aerodactylus - Jetsam Dreams - 02 Pterosoaring.mp4',
    'transition2.mp4',
    'Aerodactylus - Jetsam Dreams - 03 In Plain Flight.mp4',
    'transition3.mp4',
    'Aerodactylus - Jetsam Dreams - 04 Flutter On.mp4',
    'transition4.mp4',
    'Aerodactylus - Jetsam Dreams - 05 Wintering Grounds.mp4',
    'transition5.mp4',
    'Aerodactylus - Jetsam Dreams - 06 Alpenglow.mp4',
    'transition6.mp4',
    'Aerodactylus - Jetsam Dreams - 07 Jettison the Rest.mp4',
    'transition7.mp4',
    'Aerodactylus - Jetsam Dreams - 08 Sub Rosa Sunburst.mp4'
]

In [10]:
clips = [VideoFileClip(filename) for filename in video_files]

video = concatenate_videoclips(clips, method='compose')
video.write_videofile('Aerodactylus - Jetsam Dreams - Full Album.mp4')

Moviepy - Building video Aerodactylus - Jetsam Dreams - Full Album.mp4.
MoviePy - Writing audio in Aerodactylus - Jetsam Dreams - Full AlbumTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video Aerodactylus - Jetsam Dreams - Full Album.mp4



Moviepy - Done !
Moviepy - video ready Aerodactylus - Jetsam Dreams - Full Album.mp4
